# Retriever

To find the answer in a document, we'd first need to find relevant documents where the answer might exist. The paper suggests we use Term Frequency - Iverted Document Frequency (TF-IDF) to score documents based on a query.

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk

## Loading Data
The _data_ here is a json file with 1013 articles from Wikipedia. We're using a smaller dataset to implement and test our code before moving on to the actual wiki datadump.

In [2]:
docs = pd.read_json('../data/1000-wiki-edu-parsed.json', lines=True)

In [36]:
docs.title

0                                    Assistive technology
1                                           Alphabet song
2                    A Vindication of the Rights of Woman
3                                             Abstraction
4                                           Arthur Jensen
5                                                 Bauhaus
6                                                 Braille
7                                           B. F. Skinner
8                                                 Cooking
9                                                 College
10                                      Community college
11                                              Confucius
12                                 Charles Sanders Peirce
13                                        Comparative law
14                                            Carl Rogers
15                                             Dictionary
16                                               Dyslexia
17            

In [4]:
print(docs.iloc[0].text)

Assistive technology

Assistive technology is an umbrella term that includes assistive, adaptive, and rehabilitative devices for people with disabilities while also including the process used in selecting, locating, and using them. People who have disabilities often have difficulty performing activities of daily living (ADLs) independently, or even with assistance. ADLs are self-care activities that include toileting, mobility (ambulation), eating, bathing, dressing and grooming. Assistive technology can ameliorate the effects of disabilities that limit the ability to perform ADLs. Assistive technology promotes greater independence by enabling people to perform tasks they were formerly unable to accomplish, or had great difficulty accomplishing, by providing enhancements to, or changing methods of interacting with, the technology needed to accomplish such tasks. For example, wheelchairs provide independent mobility for those who cannot walk, while assistive eating devices can enable pe

## Term Frequency
### CountVectorizer
SkLearn offers multiple modules to vectorize text. `CountVectorizer` rakes in an array of documents, counts the occurences of every unique word in all the documents and returns a `n_docs x n_unique_words` matrix. Every row in the matrix construes a document and every column a _feature_ (word). 

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
# Array of stopwords that I gracefully borrowed from the DrQA implementation
STOPWORDS = {
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your',
    'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she',
    'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their',
    'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that',
    'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being',
    'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an',
    'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of',
    'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through',
    'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
    'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then',
    'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any',
    'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor',
    'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can',
    'will', 'just', 'don', 'should', 'now', 'd', 'll', 'm', 'o', 're', 've',
    'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven',
    'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 'weren',
    'won', 'wouldn', "'ll", "'re", "'ve", "n't", "'s", "'d", "'m", "''", "``"
}

In [7]:
vectorizer = CountVectorizer(ngram_range=(1,2), strip_accents='unicode', stop_words=STOPWORDS)
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None,
        stop_words={'mustn', 'but', 'when', 'was', 'so', 'he', 'your', 'same', 'doing', 'which', 've', 'because', 'over', 'not', 'once', 'any', 'both', 'there', 'hers', 'can', 'few', 'before', 'each', "'ll", 'down', 'about', 'with', 'against', 'then', 'won', 'their', 'this', 'had', 'theirs', 'through', 'too...'how', 'being', 'some', 's', 'is', "'s", 'ain', '``', 'aren', 'while', 'you', 'yourself', 'd', 'll'},
        strip_accents='unicode', token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

#### The actual counting
`CountVectorizer`'s `fit_transform` runs through all the documents, keeps track of every unique word and the number of times it's appeared in the document.  
`fit_transform` returns a sparse matrix. Every row is a document and every column represents a feature. `doc_counts[X][Y]` would hold the number of times the word represented by index `Y` appears in document `X`.

In [8]:
doc_counts = vectorizer.fit_transform(docs.text)
doc_counts

<1013x906821 sparse matrix of type '<class 'numpy.int64'>'
	with 1656980 stored elements in Compressed Sparse Row format>

In [10]:
doc_counts.toarray()

MemoryError: 

#### How do we know what each column means?
The feature to index map is stored in the vectorizer's `vocabulary_`. `vocabulary_[word]` would give the index of the `word` if present in the vocabulary. 

In [11]:
vectorizer.vocabulary_

{'assistive': 87860,
 'technology': 808107,
 'umbrella': 845755,
 'term': 810988,
 'includes': 410521,
 'adaptive': 43356,
 'rehabilitative': 676427,
 'devices': 243603,
 'people': 593881,
 'disabilities': 250511,
 'also': 59708,
 'including': 410961,
 'process': 631917,
 'used': 859410,
 'selecting': 726782,
 'locating': 485448,
 'using': 861820,
 'often': 563419,
 'difficulty': 247991,
 'performing': 597178,
 'activities': 41422,
 'daily': 220573,
 'living': 483985,
 'adls': 45857,
 'independently': 414836,
 'even': 300413,
 'assistance': 87609,
 'self': 727249,
 'care': 139998,
 'include': 408818,
 'toileting': 828225,
 'mobility': 525600,
 'ambulation': 64113,
 'eating': 268548,
 'bathing': 105109,
 'dressing': 262935,
 'grooming': 372516,
 'ameliorate': 64181,
 'effects': 275546,
 'limit': 479465,
 'ability': 30861,
 'perform': 596314,
 'promotes': 641396,
 'greater': 371206,
 'independence': 414442,
 'enabling': 284068,
 'tasks': 802910,
 'formerly': 341351,
 'unable': 845920,
 '

The inverse index-to-word map is returned by `get_feature_names`.

In [12]:
features = vectorizer.get_feature_names()
display(features[10000:10010])
display(len(features))

['1948 changing',
 '1948 closed',
 '1948 college',
 '1948 created',
 '1948 declared',
 '1948 declares',
 '1948 direct',
 '1948 educational',
 '1948 electrical',
 '1948 enrollment']

906821

In [13]:
vectorizer.vocabulary_['1948 birch']

9997

### Feature Hashing with `HashingVectorizer`
`HashingVectorizer` implements the `CountVectorizer` but with the hashing trick. More info on feature hashing [here](https://en.wikipedia.org/wiki/Feature_hashing). With feature hashing we lose the ability to inspect the data, since the hash function is oneway and is not invertible, in exchange for memory efficiency

The API is almost exactly the same

In [14]:
from sklearn.feature_extraction.text import HashingVectorizer

In [15]:
h_vectorizer = HashingVectorizer(ngram_range=(1,2), strip_accents='unicode', stop_words=STOPWORDS)
h_vectorizer

HashingVectorizer(alternate_sign=True, analyzer='word', binary=False,
         decode_error='strict', dtype=<class 'numpy.float64'>,
         encoding='utf-8', input='content', lowercase=True,
         n_features=1048576, ngram_range=(1, 2), non_negative=False,
         norm='l2', preprocessor=None,
         stop_words={'mustn', 'but', 'when', 'was', 'so', 'he', 'your', 'same', 'doing', 'which', 've', 'because', 'over', 'not', 'once', 'any', 'both', 'there', 'hers', 'can', 'few', 'before', 'each', "'ll", 'down', 'about', 'with', 'against', 'then', 'won', 'their', 'this', 'had', 'theirs', 'through', 'too...'how', 'being', 'some', 's', 'is', "'s", 'ain', '``', 'aren', 'while', 'you', 'yourself', 'd', 'll'},
         strip_accents='unicode', token_pattern='(?u)\\b\\w\\w+\\b',
         tokenizer=None)

In [16]:
h_doc_counts = h_vectorizer.transform(docs.text)
h_doc_counts

<1013x1048576 sparse matrix of type '<class 'numpy.float64'>'
	with 1654314 stored elements in Compressed Sparse Row format>

#### An important realisation lol
Okay so if you notice the shape of the sparse matrix returned by the `HashingVectorizer` you'd notice that there are more number of columns (2^20, the default) as compared to the one returned by `CountVectorizer`.  
```
doc_counts (returned by CountVectorizer)     => 1013 x  906821
h_doc_counts (returned by HashingVectorizer) => 1013 x 1048576
```
This seems counterproductive, since we use feature hashing to make it more memory efficient. But, if you look closely, the number of stored elements in `h_doc_counts` is lower than in `doc_counts`. We save memory since the matrices are stored in the CSR format.
```
doc_counts (returned by CountVectorizer)     => 1656980
h_doc_counts (returned by HashingVectorizer) => 1654314
```
A mere 2666 elements lesser, but that's because of how small our dataset is; A mere _1013_ documents compared to the 5,762,091 articles that we'd be using in the completed program.

## TFIDF Transform
We have the term frequency matrix, we need to convert it into a TFIDF matrix. `TfidfTransformer` does that for us

In [17]:
from sklearn.feature_extraction.text import TfidfTransformer

In [18]:
transformer = TfidfTransformer()
transformer

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [19]:
doc_tfidf = transformer.fit_transform(h_doc_counts)

In [20]:
doc_tfidf

<1013x1048576 sparse matrix of type '<class 'numpy.float64'>'
	with 1653345 stored elements in Compressed Sparse Row format>

## Query processing
Now that we have our tf-idf score matrix ready for every document, we need to process every question to return the documents with the highest score.

In [67]:
query = ["What is cooking"]

In [68]:
query_count = h_vectorizer.transform(query)

In [69]:
query_count

<1x1048576 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [70]:
query_tfidf = transformer.transform(query_count)

In [71]:
query_tfidf

<1x1048576 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [72]:
res = np.dot(query_tfidf, np.transpose(doc_tfidf))

In [73]:
res

<1x1013 sparse matrix of type '<class 'numpy.float64'>'
	with 18 stored elements in Compressed Sparse Row format>

In [74]:
res.toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

#### Finding the k largest values

In [75]:
k = 10

In [76]:
ind = np.argpartition(res.data, -k)[-k:]
ind_sort = ind[np.argsort(-res.data[ind])]

In [77]:
ind_sort

array([17,  9,  0, 10,  2,  3,  6, 16,  1, 15])

In [78]:
res.data[ind_sort]

array([0.47008715, 0.09140693, 0.06783985, 0.02418746, 0.02164328,
       0.01501773, 0.01008289, 0.00883069, 0.00855724, 0.0079207 ])

In [79]:
best_doc_indices = res.indices[ind_sort]
best_doc_indices

array([  8, 149, 769, 143, 627, 497, 200,  25, 752,  34], dtype=int32)

In [84]:
contexts = []
for index in best_doc_indices:
    contexts.append(docs.text[index])

In [85]:
contexts

['Cooking\n\nCooking or cookery is the art, technology, science and craft of preparing food for consumption. Cooking techniques and ingredients vary widely across the world, from grilling food over an open fire to using electric stoves, to baking in various types of ovens, reflecting unique environmental, economic, and cultural traditions and trends. The ways or types of cooking also depend on the skill and type of training an individual cook has. Cooking is done both by people in their own dwellings and by professional cooks and chefs in restaurants and other food establishments. Cooking can also occur through chemical reactions without the presence of heat, such as in ceviche, a traditional South American dish where fish is cooked with the acids in lemon or lime juice.\n\nPreparing food with heat or fire is an activity unique to humans. It may have started around 2 million years ago, though archaeological evidence for it reaches no more than 1 million years ago.\n\nThe expansion of a